# Interactive Data Visualization

<img style="float:left;" src="dataviz.jpg" width="500" height="500" />

### Autocomplete

In [1]:
from bqwidgets import TickerAutoComplete
from IPython.display import display

# Pick up a ticker -> Auto complete
ticker_ac = TickerAutoComplete(description='Security', yellow_keys=['Equity'], max_results=5, value='IBM US Equity')

display(ticker_ac)

TickerAutoComplete(value='IBM US Equity', description='Security', max_results=5, yellow_keys=['Equity'])

### Date Picker

In [3]:
from bqwidgets import DatePicker
from IPython.display import display

# Datepicker
date_picker = DatePicker(description='Date', date_format='%Y-%m-%d')

display(date_picker)

DatePicker(date_format='%Y-%m-%d', description='Date', value='2020-09-09')

### Dropdown

In [4]:
from ipywidgets import Dropdown
from IPython.display import display

# Dropdown
dropdown = Dropdown(options=['Annual', 'Semi-Annual','Quarterly'],value='Annual',description='Periodicity')

display(dropdown)


Dropdown(description='Periodicity', options=('Annual', 'Semi-Annual', 'Quarterly'), value='Annual')

### Textbox

In [5]:
import ipywidgets as ipw
from IPython.display import display

# Textbox
textbox = ipw.Text(value='Report Hello World', placeholder='Some text...', description='Report Name')

display(textbox)

Text(value='Report Hello World', description='Report Name', placeholder='Some text...')

### Interactive Button

In [6]:
from ipywidgets import *
from bqwidgets import *
from IPython.display import display

import bql
import pandas as pd

# Link UI and bql service
ticker_ac = TickerAutoComplete(description='Security', yellow_keys=['Equity'], max_results=5, value='IBM US Equity')
button = widgets.Button(description='Get Price', button_style='')

display(HBox([ticker_ac, button]))

def on_button_click(btn):
    ticker = ticker_ac.value.split(':')[0].strip()
    bq = bql.Service()
    response = bq.execute("get(PX_LAST) for(['{}'])".format(ticker)) #https://pyformat.info/
    print(response[0].df())
    
button.on_click(on_button_click)

In [7]:
from ipywidgets import Button
from bqwidgets import TickerAutoComplete
from IPython.display import display, clear_output

import bql
import pandas as pd

# A bit better display of the table but need more imporvement

ticker_ac = TickerAutoComplete(description='Index', yellow_keys=['Index'], max_results=5, value='INDU Index')
button = Button(description='Get Price', button_style='')

display(HBox([ticker_ac, button]))

def on_button_click(btn):
    
    ticker = ticker_ac.value.split(':')[0].strip()
    bq = bql.Service()
    response = bq.execute("get(PX_LAST, INDUSTRY_SECTOR) for(Members(['{}']))".format(ticker))
    
    data = response[0].df().reset_index()
    
    data = data.round({'PX_LAST': 2})[['ID', 'PX_LAST']]
    
    clear_output()
    display(data.head(4))
    
    data = response[1].df()
    #print(type(list(data['INDUSTRY_SECTOR'].unique())))
    display(list(data['INDUSTRY_SECTOR'].unique()))
    
button.on_click(on_button_click)

## Simple Price Chart

In [8]:
from ipywidgets import Button, HBox
from bqwidgets import TickerAutoComplete
from IPython.display import display

import bql
import pandas as pd

from bqplot import DateScale, LinearScale, Axis, Lines, Figure

DEFAULT_TICKER = 'NDX Index'

ticker_ac = TickerAutoComplete(description='Index', yellow_keys=['Index'], max_results=5, value=DEFAULT_TICKER)
button = Button(description='Get Price', button_style='')

display(HBox([ticker_ac, button]))

def getSixMonthPrice(ticker):
    bq = bql.Service()
    
    data = bq.execute("get(dropna(px_last(start=-6M))) for (['{}'])".format(ticker))
    data = data[0].df()
    
    data.rename(
        columns={"DATE": "Date", 
        "CURRENCY": "Currency", 
        data.columns[-1]: "Last Price"}, 
        inplace=True)
    
    return data


def on_button_click(btn):
    ticker = ticker_ac.value.split(':')[0].strip()
    data = getSixMonthPrice(ticker)
    line.y = data['Last Price'].values
    fig.title = '{} Price'.format(ticker)
    
button.on_click(on_button_click)

# Get data for the default ticker
data = getSixMonthPrice(DEFAULT_TICKER)

# Initialize the chart with default ticker
x_sc = DateScale()
y_sc = LinearScale()

ax_x = Axis(label='Date', scale=x_sc, grid_lines='solid')
ax_y = Axis(label='Price', scale=y_sc, orientation='vertical', grid_lines='solid')

line = Lines(x=data['Date'].values, y=data['Last Price'].values, scales={'x': x_sc, 'y': y_sc}, display_legend=True, labels=['Price']) # Note labels takes python list!!!
fig = Figure(axes=[ax_x, ax_y], marks=[line], title='{} Price'.format(DEFAULT_TICKER))




display(fig)

Figure(axes=[Axis(label='Date', scale=DateScale()), Axis(label='Price', orientation='vertical', scale=LinearSc…

# <font color = 'hotpink'>Interactivity (Callback) </font>

#### We'll define a callback function that will get triggered when the user clicks the button. In the case of this particular application, it’s ultimately going to pull down BQL data as a data frame, construct a plot, and then display that plot. 
#### Later on we will amend this callback function to pull in the BQL data. We’ll leave the basic structure in place, but with just simple tweaks we can dramatically change the behavior. For the time being we'll just use our function to use generate the canned random data.


In [8]:
#import libraries
import ipywidgets as widgets
import bqviz as bqv
import bqplot as bqp
import pandas as pd
import numpy as np

def pull_med_eps_chg(index_name, months_back):
    df = pd.DataFrame(np.random.randn(10))
    return df

def construct_bar_plot(df):
    plot = bqv.BarPlot(df)
    return plot.show()

In [9]:
#GUI
index_picker = widgets.Dropdown(options=['SPX Index','IBOV Index', 'INDU Index'])
months_back_picker = widgets.IntSlider(description='Months Back', value=3,min=1,max=24)
button = widgets.Button(description='Click to Compute',button_style='success')
output_area = widgets.HBox([])
gui = widgets.VBox([index_picker, months_back_picker, button, output_area])
gui

#### Define the callback function to do three things:
- Pull the (canned) median eps change data by passing in the index and months back stripped from the GUI widgets
- Feed the data frame into the function to construct a bar plot
- Put the bar plot into the "output area" placeholder

In [10]:
def button_callback(btn):
    df = pull_med_eps_chg(index_picker.value, months_back_picker.value)
#    p = construct_bar_plot(df)
#    output_area.children=[p]
    return df
    

### We need to tie the callback to the button, so that when the button is clicked, the chart gets produced and placed in the output_area

In [11]:
a = button.on_click(button_callback)
gui

In [12]:
a